## Grid-Search Univariate LSTM-RNN Sumatera Selatan Percobaan 1

### 1. Declaration Library

In [ ]:
# library manipulation dataset
import pandas as pd
from pandas import concat
from pandas import DataFrame
from pandas import read_csv
from pandas import read_excel

# library manipulation array
import numpy as np
from numpy import concatenate
from numpy import array

# library configuration date and time
import time
from datetime import datetime

# library data visualization
import seaborn as sns
from matplotlib import pyplot
from matplotlib import pyplot as plt

# library analysis acf and pacf
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf

# library normalize data with max-min algorithm
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

# library algorithm lstm-rnn with keras
import tensorflow as tf
from tensorflow.keras import models
from keras.models import Sequential
from keras.layers import RNN
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
from keras.optimizers import Adam, Adamax, RMSprop, SGD

# Early stoping
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

# library evaluation model
from math import sqrt
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

### 2. Akuisisi Data

In [ ]:
# set time for complexity computation
start = time.time()

In [ ]:
# fix random seed for reproducibility
np.random.seed(1234)

In [ ]:
# load dataset csv
dataset = pd.read_csv("dataset/modis_sumsel_bulanan.csv", parse_dates=['acq_date'], engine="python")

In [ ]:
# # set index
# dataset = dataset.set_index("Date");

In [ ]:
# show metadata dataset
dataset.info()

In [ ]:
# show dataset
dataset

### 3. Exploration Data Analysis

- Data Visualization

In [ ]:
# make frame
fig, ax = plt.subplots(figsize = (20,6))

# make time series plot
ax.plot(dataset["acq_date"], dataset["hotspot"], color="tab:blue", label="hotspot sumsel 2001-2020", linewidth=3)

# make are labels
ax.set_title("Hotspot Sumsel 2001-2020", fontsize=14)
ax.set_xlabel("Date", fontsize=12)
ax.set_ylabel("Sum of hostpot", fontsize=12)
ax.legend(loc='best')
ax.grid(True)

# show plot time series
plt.show()

- Analysis ACF and PACF

In [ ]:
# make frame
fig, ax= plt.subplots(nrows=1, ncols=2, facecolor="#F0F0F0", figsize = (20,5))

# plot acf
plot_acf(dataset["hotspot"], lags=24, ax=ax[0])
ax[0].grid(True)

# plot pacf
plot_pacf(dataset["hotspot"],lags=24, ax=ax[1], method="yw")
ax[1].grid(True)

# show plot acf and pacf
plt.show()

### 4. Praproses Data

- feature selection

In [ ]:
# set a feature selection
df_sumsel = dataset.filter(["hotspot"])

In [ ]:
# convert dataframe to series
df_sumsel = df_sumsel.values

In [ ]:
# show a dataset
np.round(df_sumsel[:5],7)

In [ ]:
# view a dimension dataset
df_sumsel.shape

- normalization data

In [ ]:
# normalize data with max-min algorithm
scaler = MinMaxScaler(feature_range=(-1, 1))
scaled_data = scaler.fit_transform(np.array(df_sumsel).reshape(-1,1))

In [ ]:
# show a dataset after normalize
np.round(scaled_data[:5],7)

In [ ]:
# view a dimension dataset after normalize
scaled_data.shape

In [ ]:
# make a frame
fig, ax = plt.subplots(figsize = (20,6))

# make a time series plot
ax.plot(dataset["acq_date"], scaled_data, color="tab:blue", label="hotspot sumsel 2001-2020", linewidth=2.5)

# make a labels
ax.set_title("Hotspot Sumsel Sensor MODIS 2001-2020", fontsize=14)
ax.set_xlabel("Tahun", fontsize=12)
ax.set_ylabel("Jumlah Hostpot", fontsize=12)
ax.legend(loc='best')
ax.grid(True)

# show plot time series
plt.show()

- set data train and data test

In [ ]:
# set data train
train_size = int(len(scaled_data) * 0.8)

In [ ]:
# set loc data train
train_data = scaled_data[0:train_size,:]

In [ ]:
# show data train
np.round(train_data[:5],7)

In [ ]:
# view dimension of data train
train_data.shape

In [ ]:
# set data test
test_size = len(df_sumsel) - train_size

In [ ]:
# set loc data train
test_data = scaled_data[train_size:len(df_sumsel),:]

In [ ]:
# show data train
np.round(test_data[:5],7)

In [ ]:
# view dimension of data train
test_data.shape

In [ ]:
# make a frame
fig, ax = plt.subplots(figsize = (20,7))

# make a time series plot
ax.plot(dataset.iloc[0:train_size,0], train_data, color="tab:blue", label="Data Latih", linewidth=2.5)
ax.plot(dataset.iloc[train_size:len(scaled_data),0], test_data, color="tab:red", label="Data Uji", linewidth=2.5)

# make are labels
ax.set_title("Hotspot Sumsel Sensor MODIS 2001-2020", fontsize=14)
ax.set_xlabel("Tahun", fontsize=12)
ax.set_ylabel("Jumlah Hostpot", fontsize=12)
ax.legend(loc="best")
ax.grid(True)

# show plot time series
plt.show()

### 5. Supervised Learning

In [ ]:
# function for supervised learning
def create_dataset(dataset, look_back):
    
    # declare variable X and Y
    dataX = []
    dataY = []
    
    # for loop for create supervised learning
    for i in range(look_back, len(dataset)):
        
        # insert value X and Y 
        dataX.append(dataset[i-look_back:i, 0])
        dataY.append(dataset[i, 0])
        
    # return value X and Y
    return np.array(dataX), np.array(dataY)

- process supervised learning, with look back t-1 into X=t and Y=t+1

In [ ]:
# set look back -1
look_back = 1

In [ ]:
# set supervised learning for data train
x_train, y_train = create_dataset(train_data, look_back)

In [ ]:
# view a dimension dataset after supervised learning
print(x_train.shape, y_train.shape)

In [ ]:
# set supervised learning for data test
x_test, y_test = create_dataset(test_data, look_back)

In [ ]:
# view a dimension dataset after supervised learning
print(x_test.shape, y_test.shape)

- Check data train, for result supervised learning

In [ ]:
temp_trainX = pd.DataFrame(x_train, columns=['y train'])
temp_trainY = pd.DataFrame(y_train, columns=['x train'])

In [ ]:
hasil_train = pd.concat([temp_trainX, temp_trainY], axis=1)
hasil_train.head()

In [ ]:
hasil_train.info()

- Check data test, for result supervised learning

In [ ]:
temp_testX = pd.DataFrame(x_test, columns=['y test'])
temp_testY = pd.DataFrame(y_test, columns=['x test'])

In [ ]:
hasil_test = pd.concat([temp_testX, temp_testY], axis=1)
hasil_test.head()

In [ ]:
hasil_test.info()

- reshape input to be [samples, time steps, features]

In [ ]:
# reshape data train
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [ ]:
print(x_train.shape, y_train.shape)

In [ ]:
# reshape data train
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
print(x_test.shape, y_test.shape)

### 6. Hyperparameter GridSearchCV LSTM-RNN

- library for hyperparameter

In [ ]:
import itertools
from scikeras.wrappers import KerasRegressor
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

- set models lstm-rnn

In [ ]:
# function models lstm-rnn
def get_model(activation='tanh', optimizer='Adam', dropout_rate=0.00):
    tf.keras.backend.clear_session()
    
    # design network
    model = Sequential()
    
    # first LSTM-RNN layer with dropout regularisation
    model.add(LSTM(units=10, activation=activation, input_shape=(x_train.shape[1], 1)))
    model.add(Dropout(dropout_rate))
    
    # The output layer
    model.add(Dense(1))
    
    # if optimizer == "adam":
    #     opt = Adam(learning_rate)
    # elif optimizer == "adamax":
    #     opt = Adamax(learning_rate)
    # elif optimizer == "rmsprop":
    #     opt = Adamax(learning_rate)
    # else:
    #     opt = SGD(learning_rate)
    
    # Compiling model the LSTM-RNN
    model.compile(
        optimizer=optimizer,
        loss="mae",
        metrics=[
            tf.keras.metrics.MeanAbsoluteError(),
            tf.keras.metrics.MeanSquaredError(),
            tf.keras.metrics.RootMeanSquaredError()
        ]
    )

    # return values
    return model;

Set parameter
<ul>
    <li>activation function = tanh, sigmoid, relu, selu, softplus, elu.</li>
    <li>optimizers = adam, adamax, rmsprop, sgd.</li>
    <li>droupout = 0.1, 0.2, 0.3, 0.4, 0.5</li>
    <li>batch size = 4, 8, 16, 32, 64</li>
    <li>epoch = 500, 1000, 1500, 2000</li>
</ul>

In [ ]:
# set parameter activation
activation = ['tanh', 'sigmoid', 'relu', 'selu', 'elu', 'softplus']

In [ ]:
# set parameter optimizer
optimizer = ['Adam', 'Adamax', 'RMSprop', 'SGD']

In [ ]:
# set parameter dropout_rate
dropout_rate = [0.00, 0.05, 0.1, 0.15, 0.2, 0.25]

In [ ]:
# set parameters batch sizes
batch_size = [2, 4, 8, 16, 32]

In [ ]:
# set parameter epochs
epochs = [10]

In [ ]:
# convert parameter to dictionary
param_grid = dict(
    model__activation=activation,
    model__optimizer=optimizer,
    model__dropout_rate=dropout_rate,
    batch_size=batch_size,
    epochs=epochs
)

In [ ]:
param_grid

In [ ]:
grid_model = KerasRegressor(model=get_model, verbose=1)

In [ ]:
grid_search = GridSearchCV(estimator=grid_model, param_grid=param_grid, n_jobs=-1, cv=2)

In [ ]:
grid_result = grid_search.fit(x_train, y_train)

In [ ]:
print(grid_result)

In [ ]:
# summarize results
print("Best parameters: %f using %s\n" % (grid_search.best_score_, grid_search.best_params_))

In [ ]:
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
params = grid_search.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
len(params)

### 7. Evaluasi Model LSTM-RNN

In [ ]:
# Set akhir waktu komputasi 
end = time.time()

In [ ]:
# Proses menghitung waktu komputasi
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)

In [ ]:
# Hasil waktu komputasi
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))